<a href="https://colab.research.google.com/github/explorewtxs/AI_pai_second_test/blob/main/step2/fine_tune/Bert_Fine_Tune_UsingTrainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets
!pip install scikit-learn
!pip install transformers

In [ ]:
from datasets import load_dataset
data = load_dataset("stanfordnlp/imdb")

In [ ]:
small_train_dataset = data["train"].shuffle(seed=42).select(range(5000))
small_eval_dataset = data["test"].shuffle(seed=42).select(range(5000))
data
data["train"][1000]

{'text': 'Although I have to admit I laughed more watching this movie than the last few comedies I saw.<br /><br />The budget must have consisted of pocket change from the actors. The production values are so low that they actual made it kind of fun to watch. Reminds me of the Robot Monster made up of a guy in a gorilla suit with a cardboard diving helmet on.<br /><br />In one scene a hapless victim gets their arm and leg cut off. Geez, hard to believe but the Black Knight scene from Holy Grail was more realistic. I kept wondering why the victim didn\'t start shouting " None Shall Pass" and " It\'s only a flesh wound, I\'ve had worse". It was one of the funniest scenes I\'ve seen in the past year.<br /><br />The "gladiator/demon" was a stitch too. Between the horribly cheap costume and the geeky look of the guy in it the end result was hysterical.<br /><br />Truly a movie that is bad enough to be watchable. Kind of like seeing a slow motion auto accident on film.<br /><br />',
 'label'

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
  return tokenizer(examples["text"],padding = "max_length",truncation = True)
Train = small_train_dataset.map(tokenize_function,batched=True)
Eval = small_eval_dataset.map(tokenize_function,batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score,precision_recall_fscore_support
# 定义模型评价函数
def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  '''for i in range(100):

    print("show_detail")
    print(eval_pred[0][i])
    print(labels[i])
    print(preds[i])'''
  precision,recall,f1,_ = precision_recall_fscore_support(labels,preds,average='binary')#先label后preds，不要弄反了
  acc = accuracy_score(labels,preds)
  return {
      'accuracy':acc,
      'f1':f1,
      'precision':precision,
      'recall':recall
  }

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased",num_labels = 2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir = 'result',
    eval_strategy = "epoch",
    num_train_epochs = 3,
    learning_rate = 5e-5,
    weight_decay = 0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=Train,
    eval_dataset=Eval,
    compute_metrics=compute_metrics,
)


In [ ]:
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")
print(f"F1-Score: {results['eval_f1']}")
print(f"Precision: {results['eval_precision']}")
print(f"Recall: {results['eval_recall']}")

Accuracy: 0.626
F1-Score: 0.6157007809288944
Precision: 0.6347457627118644
Recall: 0.5977653631284916


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1,Precision,Recall
1,0.440500,0.368320,0.007000,0.875400,0.885373,0.821441,0.960096
2,0.244500,0.420766,0.007000,0.901800,0.901937,0.902839,0.901038
3,0.138000,0.495680,0.007000,0.907200,0.908769,0.895736,0.922187


TrainOutput(global_step=1875, training_loss=0.23336915079752604, metrics={'train_runtime': 1913.3982, 'train_samples_per_second': 7.839, 'train_steps_per_second': 0.98, 'total_flos': 3946665830400000.0, 'train_loss': 0.23336915079752604, 'epoch': 3.0})

In [ ]:
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")
print(f"F1-Score: {results['eval_f1']}")
print(f"Precision: {results['eval_precision']}")
print(f"Recall: {results['eval_recall']}")


Accuracy: 0.9072
F1-Score: 0.9087691702713331
Precision: 0.8957364341085271
Recall: 0.9221867517956903
